In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='Blues')
from slmsuite.holography.algorithms import Hologram, SpotHologram
from tools.IonChainTools import *

In [ ]:
def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200):
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

In [ ]:
target_size = (1200,1920)
target = np.zeros(target_size, dtype=int)
pixel_size = np.array([7.8e-6,7.8e-6]) 
ions = np.array([0,1,2,5])

In [ ]:
N = 6
v = 2*π*1e6 
mag = 100 * 7.8 / 2.2
position = np.array(ion_position_cam(N,v,pixel_size,magnititude= mag))
dy = target_size[0]//8 
pos = position[ions]
position_cam = np.column_stack((pos+target_size[1]//2,np.full(pos.shape,target_size[0]//2+dy)))

# Note: For now, we'll assume the SLM and target are the same size (since they're a Fourier pair)
slm_size = target_size
print(position_cam)

In [ ]:
#Create a virtual SLM and camera
from slmsuite.hardware.slms.slm import SLM
from slmsuite.hardware.cameras.camera import Camera
from slmsuite.hardware.cameraslms import FourierSLM

wav_um = 0.730
slm = SLM (slm_size[1],slm_size[0], dx_um=7.8, dy_um=7.8, wav_um=wav_um)
camera = Camera(target_size[1],target_size[0])

slm.set_measured_amplitude_analytic(800)
fs = FourierSLM(camera, slm)

In [ ]:
position_all = np.column_stack((np.full(position.shape,target_size[0]//2+dy),position+target_size[1]//2))
mask_ij = np.zeros((1200,1920), dtype=bool)
mask_ij[600:900,400:1500] = True
position_cam_T = position_cam.T
null_point = np.array([1920//2,1200//2-300]).T
holo = SpotHologram(shape=(1200, 1920), 
            spot_vectors=position_cam_T , 
            cameraslm=fs,
            null_region = mask_ij,
            null_vectors= null_point,
            null_radius= 60
            )

holo.optimize(method = 'WGS-Kim', maxiter=50)

In [ ]:
#plot the nearfield and of SLM
holo.plot_nearfield(padded=True,cbar=True)
limit = np.array(((500,1400),(600,900)))
holo.plot_farfield(cbar=True,limits = limit,figsize=(10, 8))

In [ ]:
#plot the enhanced farfield amplitude
image = holo.extract_farfield()
amp = np.abs(image)
# norm_amp = (amp - amp.min())/(amp.max() - amp.min())
# plt.imshow(np.log(1+np.sqrt(norm_amp)))
log_scale_data = np.log(1+np.sqrt(amp))
from matplotlib import ticker
plt.figure(figsize=(8, 6))
img = plt.imshow(log_scale_data)
cb = plt.colorbar(img, label='Amplitude', format='%.2f')

# Adjust the colorbar labels to show 10^log_data format
cb.formatter = ticker.FuncFormatter(lambda x, pos: f'10^{x:.2f}')
cb.update_ticks()

plt.title('Farfield Data Visualization')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()


In [ ]:
#calculate the intensity at the positions of ions
intensity = np.abs(image)**2
cal = []
all = range(N)
pos_all = position[all]
position_all = np.column_stack((np.full(pos_all.shape,target_size[0]//2+dy),pos_all+target_size[1]//2))
for i, pos in enumerate(position_all):
    cal.append(np.sum(intensity[int(pos[0])-60:int(pos[0])+60,int(pos[1])-60:int(pos[1])+60]))

In [ ]:
#plot the intensity of the ions
plt.scatter(range(6),cal, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

In [ ]:
#calculate the fidelity
intensity = cal
background = 0

inten = np.array([ i- background for i in intensity])
all = range(N)
undesire = [ i for i in all if i not in ions]
Omega_desire = np.sqrt(inten[ions])
Omega_undesire = np.sqrt(inten[undesire])
F1 = sum(Omega_desire)/(len(ions)*max(Omega_desire))
F2 = 1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire))
F = F1*F2

print(F1)
print(F2)
print(F)